# Coding Discussion 4

## David Lopez

In [84]:
import numpy as np
import pandas as pd
import csv

In [85]:
# Get full texts for articles


with open('/Users/davidlopez/Desktop/PPOL564/coding_discussions_ppol564_fall2021/04_coding_discussion/Data/aljazeera-khashoggi.txt') as x:
    textAlJazzera = x.read()
textAlJazzera

with open('/Users/davidlopez/Desktop/PPOL564/coding_discussions_ppol564_fall2021/04_coding_discussion/Data/bbc-khashoggi.txt') as x:
    textBBC = x.read()

with open('/Users/davidlopez/Desktop/PPOL564/coding_discussions_ppol564_fall2021/04_coding_discussion/Data/breitbart-khashoggi.txt') as x:
    textBreitbart = x.read()

with open('/Users/davidlopez/Desktop/PPOL564/coding_discussions_ppol564_fall2021/04_coding_discussion/Data/cnn-khashoggi.txt') as x:
    textCNN = x.read()

with open('/Users/davidlopez/Desktop/PPOL564/coding_discussions_ppol564_fall2021/04_coding_discussion/Data/fox-khashoggi.txt') as x:
    textFOX = x.read()

In [86]:
# load stopwords and create a list
with open('/Users/davidlopez/Desktop/PPOL564/coding_discussions_ppol564_fall2021/04_coding_discussion/Data/stop_words.csv') as x:
    reader = csv.reader(x)
    stopwords = list(reader)


from itertools import chain

stopwords = list(chain.from_iterable(stopwords))


In [87]:
def tokenize(text=None):
    '''
    Splits words in text into a list of strings. Removes stopwords, punctuation, and other characters from text.
    ----------
    Input: single string of text
    Output: list of strings
'''
    text = text.lower()
    text = text.replace('.','')
    text = text.replace(',','')
    text = text.replace('?','')
    text = text.replace("''", '')
    text = text.replace(')', '')
    text = text.replace('(', '')
    text = text.replace(':', '')
    text = text.replace(';', '')
    text = text.replace('$','')
    text = text.replace('""','')
    text = text.replace('1','')
    text = text.replace('2','')
    text = text.replace('3','')
    text = text.replace('4','')
    text = text.replace('5','')
    text = text.replace('6','')
    text = text.replace('7','')
    text = text.replace('8','')
    text = text.replace('9','')
    text = text.replace('0','')
    text_list = text.split()
    text_list2 = [word for word in text_list if word not in stopwords]
    return text_list2

In [88]:
def convert_text_to_dtm(txt):
    '''
    Converts text into a document term matrix.
    ------------
    Input: string of text
    Output: document term matrix dataframe
    '''
    d = {}
    for word in tokenize(txt):
        if word in d:
            d[word][0] += 1
        else:
            d[word] = [1]
    return pd.DataFrame(d)

In [89]:
def gen_DTM(texts=None):
    '''
    Generate a document term matrix
    ----------------------
    Input: multiple strings of text
    Output: DTM dataframe
    '''
    DTM = pd.DataFrame()
    for text in texts:
        entry = convert_text_to_dtm(text)
        DTM = DTM.append(pd.DataFrame(entry),ignore_index=True,sort=True) # Row bind

    DTM.fillna(0, inplace=True) 
    return DTM

In [90]:
articles = [textAlJazzera, textBBC, textBreitbart, textCNN, textFOX]

# Create DTM dataframe with article texts
gen_DTM(articles)

,"""a","""all","""assassination","""be","""bring","""co-operator""","""distinction""","""everything","""fist-fight""","""fistfight""",...,“may,“other,“partner,“putting,“saudi,“the,“to,“we,“we're,“why
0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [101]:

# Identify values
AlJazzera = df.iloc[0].values
BBC = df.iloc[1].values
Breitbart = df.iloc[2].values
CNN = df.iloc[3].values
FOX = df.iloc[4].values

#Dictionary to be used in cosine matrix
dictionary = {
'AlJazzera': AlJazzera,
'BBC': BBC,
'Breitbart': Breitbart,
'CNN': CNN,
'FOX': FOX
}

In [105]:
def cosine(a,b):
    '''
Produce cosine similarity for two objects
        --------------
        Input: text articles
        Output: cosine similarity
        '''
    cos = np.dot(a,b)/(np.sqrt(np.dot(a,a)) * np.sqrt(np.dot(b,b)))
    return cos.round(3)

df_dict = pd.DataFrame(dictionary)


matrix = df_dict.corr(method=cosine)
# Matrix with cosine similarities by article
matrix

,AlJazzera,BBC,Breitbart,CNN,FOX
AlJazzera,1.000,0.638,0.526,0.500,0.603
BBC,0.638,1.000,0.534,0.472,0.575
Breitbart,0.526,0.534,1.000,0.325,0.496
CNN,0.500,0.472,0.325,1.000,0.491
FOX,0.603,0.575,0.496,0.491,1.000


### In this matrix, we can observe the level of similarity of words used in each of the articles. Although we can't tell much about specific bias presented, we can see from the cosines of the articles how similar they are in word choice which may convey certain implications. For example, BBC, which is regarded as having little political leaning, is most dissimilar to Breitbart and CNN, which are knowing for leaning conservative and liberal respectively.  